In [53]:
from azure.core.exceptions import HttpResponseError

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute
import json

import os
import time

from azure.ai.ml.entities import Environment

from azure.ai.ml import command
from azure.ai.ml import Input
import uuid

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

In [2]:
credential = DefaultAzureCredential()

In [4]:
# import secrets.json

with open('secrets.json') as f:
    secrets = json.load(f)

In [5]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=secrets['aml_subscription_id'],
    resource_group_name=secrets['aml_resource_group'],
    workspace_name=secrets['aml_workspace_name']
)

In [13]:
# check the computes on workspace
list(ml_client.compute.list())

[]

In [19]:
cpu_compute_target = "cpu-cluster"

In [20]:
cpu_cluster = AmlCompute(
    name=cpu_compute_target,
    # Azure ML Compute is the on-demand VM service
    type="amlcompute",
    # VM Family
    size="STANDARD_DS3_V2",
    # Minimum running nodes when there is no job running
    min_instances=0,
    # Nodes in cluster
    max_instances=4,
    # How many seconds will the node running after the job termination
    idle_time_before_scale_down=120,
    # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
    tier="Dedicated",
)

cpu_cluster.size, type(cpu_cluster)

('STANDARD_DS3_V2', azure.ai.ml.entities._compute.aml_compute.AmlCompute)

In [21]:
# Create the cluster
cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)
type(cpu_cluster)

azure.core.polling._poller.LROPoller

In [23]:
# check status of the cluster
cpu_cluster.status()

'Succeeded'

In [30]:
custom_env_name = "aml-scikit-learn"

In [73]:
pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for sklearn model pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dependencies_dir, "environment.yml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)

pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

In [74]:
registered_model_name = "credit_defaults_model"

In [75]:
job = command(
    inputs=dict(
        data=Input(
            type="uri_file",
            path="https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls",
        ),
        test_train_ratio=0.2,
        learning_rate=0.25,
        registered_model_name=registered_model_name,
    ),
    code="./src/", 
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="aml-scikit-learn@latest",
    compute="cpu-cluster",
    experiment_name="train_model_credit_default_prediction",
    display_name="credit_default_prediction",
)

In [76]:
#list(ml_client.jobs.list())

In [77]:
ml_client.create_or_update(job)

Experiment,Name,Type,Status,Details Page
train_model_credit_default_prediction,maroon_brush_p9s7vpz30x,command,Starting,Link to Azure Machine Learning studio


In [87]:
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]
online_endpoint_name

'credit-endpoint-b085ace2'

In [88]:
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()


In [89]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved.'
)

Endpoint "credit-endpoint-b085ace2" with provisioning state "Succeeded" is retrieved.


In [90]:
[(i.name, i.version, i.latest_version) for i  in ml_client.models.list()]

[('azureml_gray_lychee_2kkdd8fyrq_output_mlflow_log_model_49031481',
  None,
  None),
 ('azureml_gray_lychee_2kkdd8fyrq_output_mlflow_log_model_1292019808',
  None,
  None),
 ('credit_defaults_model', None, '2'),
 ('azureml_maroon_brush_p9s7vpz30x_output_mlflow_log_model_1663928806',
  None,
  None),
 ('azureml_maroon_brush_p9s7vpz30x_output_mlflow_log_model_378394399',
  None,
  None)]

In [91]:
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

latest_model_version

2

In [92]:
#Deploy the model to the endpoint

model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

In [94]:


# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)


In [ ]:
blue_deployment = ml_client.begin_create_or_update(blue_deployment ).result()

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample.json",
    deployment_name="blue",
)

In [98]:
[i.name for i in ml_client.online_endpoints.list()]

['credit-endpoint-b085ace2',
 'credit-endpoint-2642e8be',
 'creditdefaultmdl-endpt-d1704edf']

In [99]:
# deleting all the above datapoints

ml_client.online_endpoints.begin_delete(name='credit-endpoint-b085ace2')

In [100]:
ml_client.online_endpoints.begin_delete(name='credit-endpoint-2642e8be')
ml_client.online_endpoints.begin_delete(name='creditdefaultmdl-endpt-d1704edf')

.

Further resources: https://learn.microsoft.com/en-us/azure/machine-learning/tutorial-azure-ml-in-a-day
